<a href="https://colab.research.google.com/github/quanvu0996/compet/blob/master/turning_model_quanvh8_wids_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENVIROMENT SETTING

In [ ]:
# COLAB ONLY
from google.colab import drive
drive.mount('/content/drive')

In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import sklearn as sk
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve, auc, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, TruncatedSVD

from multiprocessing.pool import ThreadPool
from multiprocessing import Pool 
from scipy.stats import ks_2samp

from xgboost import XGBClassifier
# from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

#### Data loading

In [4]:
folder_path = '/content/drive/MyDrive/Data/colabs_data/WiDS' #Kaggle: ../input/widsdatathon2021/ #Local PC: '.'
df_train = pd.read_csv(folder_path + "/TrainingWiDS2021.csv")
df_test = pd.read_csv(folder_path + "/UnlabeledWiDS2021.csv")
print(df_train.shape)
print(df_test.shape)

(130157, 181)
(10234, 180)


#### Feature classification
Phân loại các loại nhãn dữ liệu cần xử lý


In [5]:
# Data type 
col_target = 'diabetes_mellitus'
cols_id = ['encounter_id', 'Unnamed: 0']
cols_cat = [i for i in df_train.select_dtypes('object').columns if i not in cols_id + [col_target]]
cols_num = [i for i in df_train if i not in cols_cat + cols_id + [col_target] ]
cols_ft = cols_cat + cols_num

BASELINE DATA PIPELINE
Chạy model mà không thêm feature

# DATA PROCESSING
Prepare and transform data

In [ ]:
p_cat = Pipeline([
  ('imputer', SimpleImputer(strategy='constant', fill_value='missing')) ,
  ('encode', preprocessing.OrdinalEncoder())
])
p_num = Pipeline([
  ('fillna', SimpleImputer(strategy='constant', fill_value= -10000))
])
p1 = ColumnTransformer([
  ('cat', p_cat, cols_cat),
  ('num', p_num, cols_num)
])

pipe = Pipeline([
  ('transformer', p1),
  ('predictor', XGBClassifier(n_estimators= 100, n_jobs= -1, max_depth = 5 ))
])

cv_auc = cross_val_score( pipe, df_train[cols_ft], df_train[col_target], 
                         cv = 5, scoring = 'roc_auc', verbose = 1, n_jobs = -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
print(cv_auc)

## ADD DERIVED FEATURES

# MODEL TURNING

# SUBMISSION